# Dataset Sloan Digital Sky Survey DR14
## Data Clean

In [1]:
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.pyplot as figure

In [2]:
data = pd.read_csv("./dataset.txt",sep=",")

In [3]:
data.head(5)

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,1.237650e+18,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,752,301,4,267,3.722360e+18,STAR,-0.000009,3306,54922,491
1,1.237650e+18,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,752,301,4,267,3.638140e+17,STAR,-0.000055,323,51615,541
2,1.237650e+18,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,752,301,4,268,3.232740e+17,GALAXY,0.123111,287,52023,513
3,1.237650e+18,183.870529,0.049911,17.76536,16.60272,16.16116,15.98233,15.90438,752,301,4,269,3.722370e+18,STAR,-0.000111,3306,54922,510
4,1.237650e+18,183.883288,0.102557,17.55025,16.26342,16.43869,16.55492,16.61326,752,301,4,269,3.722370e+18,STAR,0.000590,3306,54922,512


In [26]:
y=data.iloc[:,13:14]
y.values[0]=['STAR']

### mejor seria operar en la misma y, y crear un object que asigne
* STAR=1, GALAXY=2, QSO=3

In [27]:
for i in range(10000):
    if(y.values[i]==['STAR']):
        y.values[i]=['1']
    if(y.values[i]==['GALAXY']):
        y.values[i]=['2']
    if(y.values[i]==['QSO']):
        y.values[i]=['3']

y

,class
0,1
1,1
2,2
3,1
4,1
...,...
9995,2
9996,2
9997,1
9998,2


In [14]:
dummy_class = pd.get_dummies(y["class"])
y_class = y.drop(["class"],axis=1)
y_class = pd.concat([data,dummy_class],axis=1)
y_class

,objid,ra,dec,u,g,r,i,z,run,rerun,...,field,specobjid,class,redshift,plate,mjd,fiberid,GALAXY,QSO,STAR
0,1.237650e+18,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,752,301,...,267,3.722360e+18,STAR,-0.000009,3306,54922,491,0,0,1
1,1.237650e+18,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,752,301,...,267,3.638140e+17,STAR,-0.000055,323,51615,541,0,0,1
2,1.237650e+18,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,752,301,...,268,3.232740e+17,GALAXY,0.123111,287,52023,513,1,0,0
3,1.237650e+18,183.870529,0.049911,17.76536,16.60272,16.16116,15.98233,15.90438,752,301,...,269,3.722370e+18,STAR,-0.000111,3306,54922,510,0,0,1
4,1.237650e+18,183.883288,0.102557,17.55025,16.26342,16.43869,16.55492,16.61326,752,301,...,269,3.722370e+18,STAR,0.000590,3306,54922,512,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.237650e+18,131.316413,51.539547,18.81777,17.47053,16.91508,16.68305,16.50570,1345,301,...,161,5.033450e+17,GALAXY,0.027583,447,51877,246,1,0,0
9996,1.237650e+18,131.306083,51.671341,18.27255,17.43849,17.07692,16.71661,16.69897,1345,301,...,162,5.033400e+17,GALAXY,0.117772,447,51877,228,1,0,0
9997,1.237650e+18,131.552562,51.666986,18.75818,17.77784,17.51872,17.43302,17.42048,1345,301,...,162,8.222620e+18,STAR,-0.000402,7303,57013,622,0,0,1
9998,1.237650e+18,131.477151,51.753068,18.88287,17.91068,17.53152,17.36284,17.13988,1345,301,...,163,5.033400e+17,GALAXY,0.014019,447,51877,229,1,0,0


In [9]:
columns_names = data.columns.values.tolist()
columns_names
desired_columns = ["class"]

In [10]:
sublist = [x for x in columns_names if x not in desired_columns]
sublist

['objid',
 'ra',
 'dec',
 'u',
 'g',
 'r',
 'i',
 'z',
 'run',
 'rerun',
 'camcol',
 'field',
 'specobjid',
 'redshift',
 'plate',
 'mjd',
 'fiberid']

In [11]:
X = data[sublist]
X.shape

(10000, 17)

In [28]:
X

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,redshift,plate,mjd,fiberid
0,1.237650e+18,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,752,301,4,267,3.722360e+18,-0.000009,3306,54922,491
1,1.237650e+18,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,752,301,4,267,3.638140e+17,-0.000055,323,51615,541
2,1.237650e+18,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,752,301,4,268,3.232740e+17,0.123111,287,52023,513
3,1.237650e+18,183.870529,0.049911,17.76536,16.60272,16.16116,15.98233,15.90438,752,301,4,269,3.722370e+18,-0.000111,3306,54922,510
4,1.237650e+18,183.883288,0.102557,17.55025,16.26342,16.43869,16.55492,16.61326,752,301,4,269,3.722370e+18,0.000590,3306,54922,512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.237650e+18,131.316413,51.539547,18.81777,17.47053,16.91508,16.68305,16.50570,1345,301,3,161,5.033450e+17,0.027583,447,51877,246
9996,1.237650e+18,131.306083,51.671341,18.27255,17.43849,17.07692,16.71661,16.69897,1345,301,3,162,5.033400e+17,0.117772,447,51877,228
9997,1.237650e+18,131.552562,51.666986,18.75818,17.77784,17.51872,17.43302,17.42048,1345,301,3,162,8.222620e+18,-0.000402,7303,57013,622
9998,1.237650e+18,131.477151,51.753068,18.88287,17.91068,17.53152,17.36284,17.13988,1345,301,3,163,5.033400e+17,0.014019,447,51877,229


In [ ]:
## entrenamos el modelo 
# 3 metodos distintos los que mejor se consideren

In [ ]:
#recortar los datos


In [ ]:
import seaborn as sns
sns.countplot(x="class",data=data_new)
plt.show()

#### como podemos observar una relacion entre la clase y el campo, nos dice exactamente a lo que pertenece.

In [ ]:
pd.crosstab(data["field"].values,data["class"].values).plot(kind="density",figsize=(40,20))
plt.show


In [ ]:
plt.figure(figsize=(40,20))
pd.crosstab(data["plate"].values,data["class"].values).plot(kind="area",figsize=(40,20))

plt.show()


In [ ]:
plt.figure(figsize=(40,20))
pd.crosstab(data["fiberid"].values,data["class"].values).plot(kind="kde",figsize=(40,20))

plt.show()

In [ ]:
r = data[data["QSO"]!=0]
r.shape

In [ ]:
t = data[data["STAR"]!=0]
t.shape

In [ ]:
teta  =data[data["GALAXY"]!=0]
teta.shape

## metodo alternativo? inventando, tomar para:
* r =r[n column significativas]
* t =t[n column significativas]
* teta =teta[n column significativas]


In [ ]:
data1.mean()

In [ ]:
data.groupby(["QSO","GALAXY","STAR"]).mean()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(data1["QSO"], data1["z"], data1["i"], c='r', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
c = 1+ np.log2(10000)
plt.hist(data1["QSO"], bins=int(np.ceil(c))) #bins  = [0,30,60,...]
plt.xlabel("datos astronomicos")
plt.ylabel("Frecuencia")
plt.title("histograma del datos astronomicos de una star")

In [ ]:
figure, axs = plt.subplots(3,3, sharey=True,sharex=True)

data1.plot(kind="scatter", y="ra", x = "dec", ax=axs[0][0])
data1.plot(kind="scatter",y="ra", x="field", ax=axs[0][1])
data1.plot(kind="scatter",y="ra", x="fiberid", ax=axs[0][2])

data12.plot(kind="scatter",y="redshift", x="dec", ax=axs[1][0])
data12.plot(kind="scatter",y="redshift", x="field", ax=axs[1][1])
data12.plot(kind="scatter",y="redshift", x="fiberid", ax=axs[1][2])

data123.plot(kind="scatter",y="plate", x="dec", ax=axs[2][0])
data123.plot(kind="scatter",y="plate", x="field", ax=axs[2][1])
data123.plot(kind="scatter",y="plate", x="fiberid", ax=axs[2][2])
